<a href="https://colab.research.google.com/github/mmuucha/ML-01/blob/main/Loan_Prediction_classification_Neural_NetworkV2ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import numpy as np
import matplotlib.pyplot as plt

In [3]:
import tensorflow as tf
print(tf.__version__)

2.17.0


In [4]:
data = pd.read_csv('revised_loan_default_dataset.csv')


In [5]:
data.columns = data.columns.str.strip()


In [6]:
# Separate numerical and categorical columns
numerical_columns = data.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_columns = data.select_dtypes(include=['object']).columns.tolist()

In [7]:
# Handle null values
data[numerical_columns] = data[numerical_columns].fillna(data[numerical_columns].mean())  # For numerical columns
data[categorical_columns] = data[categorical_columns].fillna('Unknown')  # For categorical columns

In [8]:
# Define your features and target
try:
    X = data.drop('LoanDefault', axis=1)  # Assuming 'LoanDefault' is your target column
    y = data['LoanDefault']
except KeyError as e:
    print(f"KeyError: {e}. Available columns: {data.columns.tolist()}")

In [9]:
# Convert categorical columns to string if they are not already
X[categorical_columns] = X[categorical_columns].astype(str)

In [10]:
# Define your numerical and categorical features
numerical_features = numerical_columns
categorical_features = categorical_columns

In [11]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data.drop('LoanDefault', axis=1), data['LoanDefault'], test_size=0.2, random_state=42)


In [12]:
# Ensure 'LoanDefault' is not in your feature lists
numerical_features = [col for col in numerical_columns if col != 'LoanDefault']
categorical_features = [col for col in categorical_columns if col != 'LoanDefault']

In [13]:
# Create a preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features) # Set handle_unknown to 'ignore'
    ]
)

In [14]:

# Fit the preprocessor on the training data and transform it
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

In [15]:
# Model creation
model_1 = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train_processed.shape[1],)),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    # Compile the model
model_1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                    loss='binary_crossentropy',
                    metrics=['accuracy'])

    # Fit the model
model_1.fit(X_train_processed, y_train, epochs=3, batch_size=32, validation_split=0.2)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/3
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 89s 43ms/step - accuracy: 0.8982 - loss: 0.3475 - val_accuracy: 0.8984 - val_loss: 0.3292
Epoch 2/3
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 139s 42ms/step - accuracy: 0.9421 - loss: 0.1553 - val_accuracy: 0.6810 - val_loss: 0.6628
Epoch 3/3
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 143s 43ms/step - accuracy: 0.9999 - loss: 3.1833e-04 - val_accuracy: 0.8790 - val_loss: 0.3946


In [16]:
model_1.evaluate(X_test_processed, y_test)

625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8882 - loss: 0.3892


[0.384555459022522, 0.8898000121116638]

still crap

In [18]:
# Prepare the new data
new_data = pd.DataFrame({
    'Date': ['2023-01-01'],
    'Income': [2000],
    'LoanAmount': [200000],
    'CreditScore': [580],
    'Age': [30],
    'LoanDuration': [30],
    'EmploymentStatus': ['Unmployed'],
    'MaritalStatus': ['Single'],
    'EducationLevel': ['Highschool']
})

# Transform the new data
X_new = preprocessor.transform(new_data)

# Make predictions
predictions = model_1.predict(X_new)

# Interpret the predictions
predicted_classes = (predictions > 0.5).astype(int)  # Binarizing predictions
print("Predicted classes:", predicted_classes)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
Predicted classes: [[0]]
